In [ ]:
"""
04_ECL_calculation.py

Calculates Expected Credit Loss (ECL) based on Probability of Default (PD), Exposure at Default (EAD),
and Loss Given Default (LGD). Also analyzes ECL distribution by credit score band.
"""


In [5]:
import pandas as pd

df = pd.read_csv('../data/model_outputs.csv', parse_dates=['OriginationDate', 'DefaultDate'])


In [6]:
print(df.columns)


df['ECL'] = df['PD'] * df['LGD'] * df['EAD']


Index(['LoanID', 'CustomerAge', 'Income', 'LoanAmount', 'InterestRate',
       'TermMonths', 'CreditScore', 'OriginationDate', 'Defaulted',
       'DefaultDate', 'DTI', 'LTI', 'AgeGroup', 'TimeToDefaultMonths',
       'OriginationMonth', 'GDP_Growth', 'Unemployment_Rate', 'Inflation_Rate',
       'GDP_Growth_3MAvg', 'Unemployment_3MAvg', 'Inflation_3MAvg', 'PD',
       'LGD', 'EAD'],
      dtype='object')


In [7]:
total_ecl = df['ECL'].sum()
print(f"Total Expected Credit Loss: €{total_ecl:,.2f}")


Total Expected Credit Loss: €7,574,342.61


In [9]:
df['CreditScoreBand'] = pd.cut(df['CreditScore'], bins=[300, 580, 670, 740, 800, 850], 
                                labels=['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'])

ecl_by_band = df.groupby('CreditScoreBand', observed=False)['ECL'].sum().sort_values(ascending=False)
print(ecl_by_band)


CreditScoreBand
Poor         3.666954e+06
Fair         1.110823e+06
Very Good    9.315211e+05
Good         9.227064e+05
Excellent    9.175751e+05
Name: ECL, dtype: float64


In [10]:
# Assign simplified staging based on PD thresholds
def assign_stage(pd):
    if pd < 0.02:
        return 'Stage 1'
    elif pd < 0.4:
        return 'Stage 2'
    else:
        return 'Stage 3'

df['IFRS9_Stage'] = df['PD'].apply(assign_stage)

ecl_by_stage = df.groupby('IFRS9_Stage')['ECL'].sum()
print(ecl_by_stage)


IFRS9_Stage
Stage 1    7.192070e+03
Stage 2    3.757725e+06
Stage 3    3.809425e+06
Name: ECL, dtype: float64


In [11]:
df.to_csv('../data/ecl_results.csv', index=False)
